# Assignment #1 Solution

## Step 1: Load and select the Philadelphia data

In [1]:
import pandas as pd

In [2]:
path = "./data/Zip_Zhvi_AllHomes.csv"
data = pd.read_csv(path, engine='python')

In [3]:
data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11
0,61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,171600.0,171600.0,171400.0,...,1130500,1123700,1119500,1116900,1110100,1098400,1086900,1080500,1072200,1064000
1,84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,158400.0,159700.0,160700.0,...,351600,352900,351900,350400,348700,347800,348200,349500,351500,354000
2,61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,347900.0,349600.0,351100.0,...,1516000,1497900,1497800,1504600,1489900,1463300,1438800,1411600,1389900,1380100
3,91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,210400.0,212200.0,212200.0,...,326600,330400,332600,334500,335800,336900,338200,338400,336900,336000
4,84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,192500.0,194500.0,196100.0,...,429000,430400,429500,428600,428700,430600,431900,430900,430900,433200


To select Philadelphia data, we have to select by the `City` column and the `State` column.

In [4]:
selection = (data['City'] == 'Philadelphia') & (data['State'] == 'PA')
data = data.loc[selection]

In [5]:
data.head()

,RegionID,RegionName,City,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,...,2018-02,2018-03,2018-04,2018-05,2018-06,2018-07,2018-08,2018-09,2018-10,2018-11
214,65779,19111,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,215,84900.0,84700.0,84500.0,...,174300,175500,175700,175800,176400,177300,178400,180300,182800,184800
300,65791,19124,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,301,43100.0,43000.0,42900.0,...,86200,88000,88900,89900,90800,91400,91900,92800,94200,95700
377,65787,19120,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,378,46100.0,46100.0,46100.0,...,91100,92100,92400,92400,92600,92800,92800,92900,93500,94200
542,65815,19148,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,543,41100.0,41100.0,41000.0,...,205700,209500,211800,213400,214900,215500,215700,216900,219800,222500
690,65812,19145,Philadelphia,PA,Philadelphia-Camden-Wilmington,Philadelphia County,691,41000.0,41000.0,41000.0,...,204100,208400,209500,209100,209600,211100,212600,213800,215100,216700


## Step 2: Calculate the annual ZHVI per ZIP code

First, remove unnecessary columns from the data frame.

In [6]:
data = data.drop(['RegionID', 'City', "State", 'Metro', 'CountyName', 'SizeRank'], axis=1)

Next, melt the data frame into long format.

In [7]:
melted = data.melt(id_vars=["RegionName"], value_name='ZHVI', var_name='Date')

In [8]:
melted

,RegionName,Date,ZHVI
0,19111,1996-04,84900.0
1,19124,1996-04,43100.0
2,19120,1996-04,46100.0
3,19148,1996-04,41100.0
4,19145,1996-04,41000.0
...,...,...,...
9787,19129,2018-11,230000.0
9788,19118,2018-11,559500.0
9789,19102,2018-11,361400.0
9790,19127,2018-11,224900.0


Convert the `Date` from a string to a pandas `Datetime` object, and then extract the year from the data and store it as the `Year` column.

In [9]:
melted['Date'] = pd.to_datetime(melted['Date'])
melted['Year'] = melted['Date'].dt.year

To calculate the annual ZHVI: 

- group by ZIP code (`RegionName`) and year (`Year`)
- take the average of the `ZVHI` column for each group

In [10]:
annual_ZHVI = melted.groupby(['RegionName', 'Year'])['ZHVI'].mean()

In [11]:
annual_ZHVI.head()

RegionName  Year
19102       1996     79966.666667
            1997     83166.666667
            1998     92550.000000
            1999    114358.333333
            2000    145175.000000
Name: ZHVI, dtype: float64

Convert from a Series object to a DataFrame by resetting the index.

In [12]:
annual_ZHVI = annual_ZHVI.reset_index()

In [13]:
annual_ZHVI.head()

,RegionName,Year,ZHVI
0,19102,1996,79966.666667
1,19102,1997,83166.666667
2,19102,1998,92550.000000
3,19102,1999,114358.333333
4,19102,2000,145175.000000


## Step 3: Find the ZIP codes with max ZHVI for each year

We need to calculate a quantity for each year, so the first step is to group the data by the `Year` column.

In [14]:
year_grps = annual_ZHVI.groupby('Year')
print(year_grps)

Next, we'll define a function that calculates the ZIP code with the max ZHVI for each year's data.

In [15]:
def get_max_ZHVI_zip(grp):
    
    # this will give you the index of the row that has
    # the max ZHVI value
    idx_max = grp['ZHVI'].idxmax()
    
    # return the RegionName in the row with the max ZVHI
    return grp.loc[idx_max, ["RegionName", "ZHVI"]]

In [16]:
max_ZIPs = year_grps.apply(get_max_ZHVI_zip)

In [17]:
max_ZIPs

,RegionName,ZHVI
Year,,
1996,19118.0,182400.000000
1997,19118.0,184358.333333
1998,19118.0,187950.000000
1999,19118.0,207350.000000
2000,19118.0,239900.000000
2001,19118.0,274108.333333
2002,19118.0,311075.000000
2003,19103.0,353258.333333
2004,19118.0,411558.333333
